In [15]:
import os

parent_dir = os.getcwd()
src_path = parent_dir

In [ ]:
def get_files(path, file_endings=['.py', '.ipynb'], ignore=[]):
    """
    Recursively walks through a directory and collects file contents.

    Args:
        path (str): The starting path (file or directory).
        file_endings (list, optional): List of file extensions to include.
        ignore (list, optional): List of directory/file names to ignore.

    Returns:
        dict or str or None: A nested dictionary representing the directory
        structure and file contents, or the content of a single file, or
        None if the file should be ignored.
    """
    # Check if the current file/directory should be ignored
    if os.path.basename(path) in ignore:
        return None

    # If it's a directory, recurse through its children
    if os.path.isdir(path):
        cache = {}
        for item_name in sorted(os.listdir(path)):
            new_path = os.path.join(path, item_name)
            result = get_files(new_path, file_endings, ignore)
            if result is not None:  # Only add if not ignored
                cache[item_name] = result
        # Return the dictionary only if it's not empty
        return cache if cache else None

    # If it's a file, check its extension
    if os.path.splitext(path)[-1] in file_endings:
        try:
            with open(path, 'r', encoding='utf-8') as f:
                return f.read()
        except Exception as e:
            return f"Error reading file: {e}"
            
    # Ignore files that don't match the desired endings
    return None

def create_tree(cache, prefix=""):
    """
    Generates a string representation of the directory tree.
    """
    if not isinstance(cache, dict):
        return ""
        
    tree_str = ""
    entries = sorted(cache.keys())
    for i, key in enumerate(entries):
        is_last = i == (len(entries) - 1)
        connector = "└── " if is_last else "├── "
        tree_str += f"{prefix}{connector}{key}\n"
        
        if isinstance(cache[key], dict):
            new_prefix = prefix + ("    " if is_last else "│   ")
            tree_str += create_tree(cache[key], new_prefix)
            
    return tree_str

def heading(title):
    """
    Creates a formatted heading string.
    """
    h_str = f"#     {title}     #"
    fill_str = "#" * len(h_str)
    return f"\n\n{fill_str}\n{h_str}\n{fill_str}\n"

def create_file_out(cache, key=None):
    """
    Recursively generates a string with the content of all files.
    """
    out_str = ""
    if isinstance(cache, dict):
        for sub_key in sorted(cache.keys()):
            out_str += create_file_out(cache[sub_key], sub_key)
    elif isinstance(cache, str) and key:
        out_str += f"{heading(key)}\n{cache.strip()}\n"
    return out_str

In [18]:
cache = get_files(src_path)
tree_str = create_tree(cache)

out_str = f"{heading('Folder Tree')}\n{tree_str}\n"

out_str += create_file_out(cache)

with open('project_file.txt', 'w') as f:
    f.write(out_str)